In [1]:
from utils import * 
import numpy as np

URL = "http://mtg.upf.edu/static/datasets/last.fm/lastfm-dataset-1K.tar.gz"
f_name = "lastfm-dataset-1K.tar.gz"
dir_name = "lastfm-dataset-1K"
dataset_f_name = "userid-timestamp-artid-artname-traid-traname.tsv"

In [2]:
download_dataset(URL, f_name)

Dataset Exists...


In [3]:
data = load_dataset(dir_name, dataset_f_name)

Read File...


In [4]:
data[:10]

,userid,timestamp,artist-id,artist-name,track-id,track-name
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)
5,user_000001,2009-05-04T13:38:31Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,To Stanford (Live_2009_4_15)
6,user_000001,2009-05-04T13:33:28Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Improvisation (Live_2009_4_15)
7,user_000001,2009-05-04T13:23:45Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Glacier (Live_2009_4_15)
8,user_000001,2009-05-04T13:19:22Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Parolibre (Live_2009_4_15)
9,user_000001,2009-05-04T13:13:38Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Bibo No Aozora (Live_2009_4_15)


Forming an artist, track id-name mapping.

In [5]:
artists = data[["artist-id", "artist-name"]].drop_duplicates()
track = data[["track-id", "track-name"]].drop_duplicates()

Forming a mapping from artist to track

In [6]:
artist_track = data[["track-id", "artist-id"]].drop_duplicates()

Playcount matrix

In [7]:
playcount = data.groupby(["userid", "track-id"]).size().reset_index()
playcount.rename(columns = {0: 'playcount'}, inplace = True)

In [ ]:
playcount[:10]

,userid,track-id,playcount
0,user_000001,00237585-8e04-4cd5-a785-f2185492ab0b,2
1,user_000001,0024d72c-136f-49f2-9078-ce4b39b94d3f,4
2,user_000001,0025055f-39c3-43e2-b874-3bf42bbc9212,2
3,user_000001,002e254d-4624-49f4-b78a-b40711b9e4f3,3
4,user_000001,00b07689-ec4c-4773-94ce-06f3d198431e,4
5,user_000001,00b811a4-762b-492e-b5ef-9a673a55da57,5
6,user_000001,00d3fbc3-4a18-42fe-a14c-6101eee7b88c,2
7,user_000001,00de4a37-1faf-49bb-a470-b8f1030e60ec,1
8,user_000001,00e2a7aa-6028-49fa-8b5d-31afe96245cc,7
9,user_000001,00f7868e-6380-4a08-8133-1b4687453699,1


In [ ]:
from scipy.sparse import csr_matrix

playcounts = playcount.pivot(index = 'userid', columns = 'track-id', values = 'playcount').fillna(0.0001)
plays_matrix = csr_matrix(playcounts.values)

In [ ]:
playcounts[:20]

Cleaning infrequent data (TODO)

## Conversion of Playcounts to Ratings

Based on work in A hybrid online-product recommendation system: Combining implicit
rating-based collaborative filtering and sequential pattern analysis by Choi et al.

Absolute Preference: AP(u, i) = ln((n(u played i)/n(u played anything)) + 1)

Relative Preference: RP(u, i) = AP(u,i) / Max_{c \in U} AP(c, i)

Implicit Rating: R(u, i) = Round up(5 * RP (u,i))

In [ ]:
# List of number of songs ever played by each user. Reduces computation
user_plays = plays_matrix.sum(axis=1) # Sum all columns
print(user_plays[:20])
print(user_plays.shape)

In [ ]:
# Compute Absolute Rating
absolute_preferences = np.log(plays_matrix / user_plays + 1)

In [ ]:
absolute_preferences.shape

In [ ]:
# 2. List of maximum ratings of each item
max_ratings = absolute_preferences.max(axis=0) # Along columns
print(max_ratings[:10])
print(max_ratings.shape)

In [ ]:
print(max_ratings[max_ratings ==0].size)

In [ ]:
print(user_plays[user_plays == 0].size)

In [ ]:
print(max_ratings.shape)

In [ ]:
# Relative Preference: RP(u, i) = AP(u,i) / Max_{c \in U} AP(c, i)
relative_preferences = absolute_preferences / (max_ratings)
print(absolute_preferences.shape)

In [ ]:
# Implicit Rating: R(u, i) = Round up(5 * RP (u,i))
implicit_ratings = np.ceil(5 * relative_preferences)

In [ ]:
implicit_ratings.max(axis=1)

In [ ]:
implicit_ratings.shape